In [8]:
# from resnet_18_fcn import *
from resnet_34_fcn import *
import time
from torch.utils.data import DataLoader
import torch
import gc
import voc
import torchvision.transforms as standard_transforms
import util
import numpy as np
import sys
import math
import copy

class MaskToTensor(object):
    def __call__(self, img):
        return torch.from_numpy(np.array(img, dtype=np.int32)).long()

# Initialize Weights with Xavier Weight Initialization
def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        torch.nn.init.xavier_uniform_(m.weight.data)
        if m.bias is not None:  # Check if the bias exists
            torch.nn.init.normal_(m.bias.data)  # Xavier not applicable for biases

In [9]:
mean_std = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
input_transform = standard_transforms.Compose([
        standard_transforms.ToTensor(),
        standard_transforms.Normalize(*mean_std)
    ])
target_transform = MaskToTensor()

train_dataset =voc.VOC('train', transform=input_transform, target_transform=target_transform)
val_dataset = voc.VOC('val', transform=input_transform, target_transform=target_transform)
test_dataset = voc.VOC('test', transform=input_transform, target_transform=target_transform)

train_loader = DataLoader(dataset=train_dataset, batch_size= 16, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size= 16, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size= 16, shuffle=False)

epochs = 20
n_class = 21

fcn_model = FCN_ResNet34(n_class=n_class)
fcn_model.apply(init_weights)

FCN_ResNet34(
  (encoder): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=

In [10]:
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

optimizer = torch.optim.Adam(fcn_model.parameters(), lr=5e-4)

# Choose an appropriate loss function from https://pytorch.org/docs/stable/_modules/torch/nn/modules/loss.html
criterion = nn.CrossEntropyLoss()
fcn_model = fcn_model.to(device)
max_model = fcn_model

earlystop = 3

In [11]:
def train():
    """
    Train a deep learning model using mini-batches.

    - Perform forward propagation in each epoch.
    - Compute loss and conduct backpropagation.
    - Update model weights.
    - Evaluate model on validation set for mIoU score.
    - Save model state if mIoU score improves.
    - Implement early stopping if necessary.

    Returns:
        None.
    """

    patience = 0
    best_iou_score = 0.0
    losses = []

    for epoch in range(epochs):
        ts = time.time()
        for iter, (inputs, labels) in enumerate(train_loader):
            
            optimizer.zero_grad() # reset optimizer gradients

            inputs, labels = inputs.to(device), labels.to(device) # both inputs and labels in device as model

            outputs = fcn_model(inputs) #  Compute outputs. Automatically in the same device as the model's

            loss = criterion(outputs, labels) #Calculate loss

            loss.backward() # Bckpropagate model

            optimizer.step() # Update the weights
            
            losses.append(loss.item())

            #if iter % 10 == 0:
            #    print("epoch{}, iter{}, loss: {}".format(epoch, iter, loss.item()))

        print("Finish epoch {}, time elapsed {}".format(epoch, time.time() - ts))
        print("Train Avg Loss: {}".format(np.mean(losses)))

        current_miou_score = val(epoch)

        # Save current IoU if better than stored best
        if current_miou_score > best_iou_score:
            best_iou_score = current_miou_score
            patience = 0
            max_model = copy.deepcopy(fcn_model) # save the best model
        else:
            patience += 1
            
        # Early stop if patience level is met
        if patience >= earlystop:
            print("Early stop at epoch " + str(epoch))
            break
    

In [12]:
def val(epoch):
    """
    Validate the deep learning model on a validation dataset.

    - Set model to evaluation mode. DONE
    - Disable gradient calculations. DONE
    - Iterate over validation data loader:
        - Perform forward pass to get outputs.
        - Compute loss and accumulate it.
        - Calculate and accumulate mean Intersection over Union (IoU) scores and pixel accuracy.
    - Print average loss, IoU, and pixel accuracy for the epoch.
    - Switch model back to training mode.

    Args:
        epoch (int): The current epoch number.

    Returns:
        tuple: Mean IoU score and mean loss for this validation epoch.
    """
    fcn_model.eval() # Put in eval mode (disables batchnorm/dropout) !
    
    losses = []
    mean_iou_scores = []
    accuracy = []
    
    with torch.no_grad(): # we don't need to calculate the gradient in the validation/testing

        # Iterate through Validation Set
        for iter, (input, label) in enumerate(val_loader):
            # label = (16, 224, 224) / batch size 16 of 244*244 masks
            # output = (16, 21, 224, 224) / batch size 16 of 21 possible classes of 244*244 masks
            
            input, label = input.to(device), label.to(device) # both inputs and labels in device as model
            
            output = fcn_model.forward(input) # Perform forward pass to get outputs.
            N, numClass, H, W = output.shape

            prediction = output.view(N, n_class, -1).argmax(dim=1).view(N, H, W) # Find the prediction for each pixel
            
            loss = criterion(output, label) # Compute loss and accumulate it.
            losses.append(loss.item())
            
            meanIOU = util.iou(prediction, label, n_class) # Calculate Intersection over Union (IoU) scores
            mean_iou_scores.append(meanIOU)

            acc = util.pixel_acc(prediction, label) # Calculate pixel accuracy
            accuracy.append(acc)
    
    print(f"Validation Loss: {np.mean(losses)}")
    print(f"Validation IoU: {np.mean(mean_iou_scores)}")
    print(f"Validation Pixel Acc: {np.mean(accuracy)}")
    print("\n")

    fcn_model.train() #TURNING THE TRAIN MODE BACK ON TO ENABLE BATCHNORM/DROPOUT!!

    return np.mean(mean_iou_scores)

def modelTest():
    """
    Test the deep learning model using a test dataset.

    - Load the model with the best weights.
    - Set the model to evaluation mode.
    - Iterate over the test data loader:
        - Perform forward pass and compute loss.
        - Accumulate loss, IoU scores, and pixel accuracy.
    - Print average loss, IoU, and pixel accuracy for the test data.
    - Switch model back to training mode.

    Returns:
        None. Outputs average test metrics to the console.
    """

    fcn_model = copy.deepcopy(max_model) # Asssume model loaded with the best weights.
    
    fcn_model.eval()  # Put in eval mode (disables batchnorm/dropout) !

    losses = []
    mean_iou_scores = []
    accuracy = []

    with torch.no_grad():  # we don't need to calculate the gradient in the validation/testing

        # Iterate through Test Set
        for iter, (input, label) in enumerate(test_loader):

            input, label = input.to(device), label.to(device) # both inputs and labels in device as model

            output = fcn_model.forward(input) # Perform forward pass to get outputs.
            N, numClass, H, W = output.shape

            prediction = output.view(N, n_class, -1).argmax(dim=1).view(N, H, W) # Find the prediction for each pixel
            
            loss = criterion(output, label) # Compute loss and accumulate it.
            losses.append(loss.item())
            
            meanIOU = util.iou(prediction, label, n_class) # Calculate Intersection over Union (IoU) scores
            mean_iou_scores.append(meanIOU)

            acc = util.pixel_acc(prediction, label) # Calculate pixel accuracy
            accuracy.append(acc)

    print(f"Test Loss at Test: {np.mean(losses)}")
    print(f"Test IoU at Test: {np.mean(mean_iou_scores)}")
    print(f"Test Pixel acc at Test: {np.mean(accuracy)}")

    fcn_model.train()  #TURNING THE TRAIN MODE BACK ON TO ENABLE BATCHNORM/DROPOUT!!

In [13]:
def exportModel(inputs):    
    """
    Export the output of the model for given inputs.

    - Set the model to evaluation mode.
    - Load the model with the best saved weights.
    - Perform a forward pass with the model to get output.
    - Switch model back to training mode.

    Args:
        inputs: Input data to the model.

    Returns:
        Output from the model for the given inputs.
    """

    fcn_model.eval() # Put in eval mode (disables batchnorm/dropout) !
    
    saved_model_path = "Fill Path To Best Model"
    # TODO Then Load your best model using saved_model_path
    
    inputs = inputs.to(device)
    
    output_image = fcn_model(inputs)
    
    fcn_model.train()  #TURNING THE TRAIN MODE BACK ON TO ENABLE BATCHNORM/DROPOUT!!
    
    return output_image

# if __name__ == "__main__":

#     val(0)  # show the accuracy before training
#     train()
#     modelTest()

#     # housekeeping
#     gc.collect()
#     torch.cuda.empty_cache()

In [14]:
train()

Finish epoch 0, time elapsed 28.398324251174927
Train Avg Loss: 3.3141595806394304
Validation Loss: 3.03933915070125
Validation IoU: 0.0006986148281381277
Validation Pixel Acc: 0.004875943820608601


Finish epoch 1, time elapsed 28.197465181350708
Train Avg Loss: 3.2375641890934537
Validation Loss: 2.990268247468131
Validation IoU: 0.00038378205046383373
Validation Pixel Acc: 0.0047214519178207


Finish epoch 2, time elapsed 32.72592353820801
Train Avg Loss: 3.1865850119363692
Validation Loss: 2.892855797495161
Validation IoU: 0.00038159010547670426
Validation Pixel Acc: 0.004688265163766399


Finish epoch 3, time elapsed 27.961517095565796
Train Avg Loss: 3.143535396882466
Validation Loss: 2.7838507890701294
Validation IoU: 0.0005183367858050099
Validation Pixel Acc: 0.004759265243485787


Early stop at epoch 3


In [15]:
modelTest()

Test Loss at Test: 2.8496783460889543
Test IoU at Test: 0.0004603377634726568
Test Pixel acc at Test: 0.006223895459411442


## Q4.a COSINE ANNEALING LR

In [16]:
fcn_model = FCN_ResNet34(n_class=n_class)
fcn_model.apply(init_weights)
fcn_model = fcn_model.to(device)

earlystop = 3
max_model = fcn_model

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(fcn_model.parameters(), lr=1e-3)

#Test Cosine Annealing Learning Rate
iterMax = math.floor(len(train_dataset)/16)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=iterMax, eta_min=1e-4)

In [17]:
def train1():
    patience = 0
    best_iou_score = 0.0
    losses = []

    for epoch in range(epochs):
        ts = time.time()
        for iter, (inputs, labels) in enumerate(train_loader):
            
            optimizer.zero_grad() # reset optimizer gradients

            inputs, labels = inputs.to(device), labels.to(device) # both inputs and labels in device as model

            outputs = fcn_model(inputs) #  Compute outputs. Automatically in the same device as the model's

            loss = criterion(outputs, labels) #Calculate loss

            loss.backward() # Bckpropagate model

            optimizer.step() # Update the weights
            
            losses.append(loss.item())
            
            scheduler.step() # For cosine annealing learning rate

        print("Finish epoch {}, time elapsed {}".format(epoch, time.time() - ts))
        print("Train Avg Loss: {}".format(np.mean(losses)))
        
        current_miou_score = val(epoch)

        if current_miou_score > best_iou_score:
            best_iou_score = current_miou_score
            patience = 0
            max_model = copy.deepcopy(fcn_model)
            # save the best model
        else:
            patience += 1
            
        # Early stop if patience level is met
        if patience >= earlystop:
            print("Early stop at epoch " + str(epoch))
            break
    

In [18]:
train1()

Finish epoch 0, time elapsed 27.06205725669861
Train Avg Loss: 3.0580615997314453
Validation Loss: 2.985409906932286
Validation IoU: 0.0009069333400389655
Validation Pixel Acc: 0.00509323253575984


Finish epoch 1, time elapsed 27.714810371398926
Train Avg Loss: 3.0240070138658797
Validation Loss: 2.7795808826174055
Validation IoU: 0.001623848682575847
Validation Pixel Acc: 0.021768339729865162


Finish epoch 2, time elapsed 28.089276790618896
Train Avg Loss: 2.9676462695712136
Validation Loss: 2.654994079044887
Validation IoU: 0.0016034590855567704
Validation Pixel Acc: 0.0218048184675656


Finish epoch 3, time elapsed 28.080803871154785
Train Avg Loss: 2.935958364180156
Validation Loss: 2.6019285917282104
Validation IoU: 0.0022127949318165
Validation Pixel Acc: 0.02381728814572704


Finish epoch 4, time elapsed 35.73695111274719
Train Avg Loss: 2.8975371973855153
Validation Loss: 2.727135385785784
Validation IoU: 0.0028959634781005883
Validation Pixel Acc: 0.03292109987131014


Finis

In [19]:
modelTest()

Test Loss at Test: 2.6762146098273143
Test IoU at Test: 0.039501374514320335
Test Pixel acc at Test: 0.6206669487689048


## Q4.b Image Transformation

In [20]:
fcn_model = FCN_ResNet34(n_class=n_class)
fcn_model.apply(init_weights)
fcn_model = fcn_model.to(device)

earlystop = 3
max_model = fcn_model

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(fcn_model.parameters(), lr=1e-3)
#Test Cosine Annealing Learning Rate
iterMax = math.floor(len(train_dataset)/16)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=iterMax, eta_min=1e-4)

In [21]:
train1()

Finish epoch 0, time elapsed 32.79289436340332
Train Avg Loss: 3.67662969657353
Validation Loss: 3.7527910130364552
Validation IoU: 0.005163105983755885
Validation Pixel Acc: 0.047436470053981404


Finish epoch 1, time elapsed 31.284151077270508
Train Avg Loss: 3.601178765296936
Validation Loss: 3.3133995703288486
Validation IoU: 0.0037010700310232724
Validation Pixel Acc: 0.053846585020727034


Finish epoch 2, time elapsed 28.172730207443237
Train Avg Loss: 3.5426324038278487
Validation Loss: 3.2292477743966237
Validation IoU: 0.0036933035855174446
Validation Pixel Acc: 0.054086365991709184


Finish epoch 3, time elapsed 28.952410459518433
Train Avg Loss: 3.499010588441576
Validation Loss: 3.4081246171678816
Validation IoU: 0.0043909126432231955
Validation Pixel Acc: 0.06435400960049198


Early stop at epoch 3


In [22]:
modelTest()

Test Loss at Test: 3.4051227058683122
Test IoU at Test: 0.005380319448185386
Test Pixel acc at Test: 0.06445597212099126


## Q4.c Weight Imbalance + (Image Transformation)

In [23]:
def getClassWeights():
    ans = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    
    # Iterate through the training set
    for iter, (inputs, labels) in enumerate(train_loader):  
        unique_elements, counts = torch.unique(labels, return_counts=True)
        
        # Count number of each class
        for i in range(len(unique_elements)):
            ans[unique_elements[i]] += counts[i]

    normalized = [tensor.tolist() for tensor in ans]
    #normalized = [num/sum(normalized) for num in normalized]
    normalized = [1/(1-pow(0.1,num/50000)) for num in normalized]
    return torch.tensor(normalized)

In [24]:
class_weights = getClassWeights()
print(class_weights)
class_weights = class_weights.to(device)

tensor([1.0000, 1.1216, 1.1572, 1.0170, 1.1338, 1.0409, 1.0001, 1.0339, 1.0002,
        1.0062, 1.0286, 1.0025, 1.0014, 1.0195, 1.0272, 1.0000, 1.3044, 1.5265,
        1.0004, 1.0040, 1.0026])


In [25]:
fcn_model = FCN_ResNet34(n_class=n_class)
fcn_model.apply(init_weights)
fcn_model = fcn_model.to(device)

earlystop = 3
max_model = fcn_model

optimizer = torch.optim.Adam(fcn_model.parameters(), lr=5e-3)
criterion = nn.CrossEntropyLoss(weight=class_weights)
#Test Cosine Annealing Learning Rate
iterMax = math.floor(len(train_dataset)/16)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=iterMax, eta_min=1e-4)

In [26]:
train1()

Finish epoch 0, time elapsed 30.08182120323181
Train Avg Loss: 2.7990044014794484
Validation Loss: 2.7985623053142
Validation IoU: 0.0005435082912659244
Validation Pixel Acc: 0.008267772441007652


Finish epoch 1, time elapsed 33.21758794784546
Train Avg Loss: 2.709486757005964
Validation Loss: 2.300388370241438
Validation IoU: 0.0014613871129413602
Validation Pixel Acc: 0.009130930553024778


Finish epoch 2, time elapsed 26.58816695213318
Train Avg Loss: 2.562353809674581
Validation Loss: 1.9935259478432792
Validation IoU: 0.055795378438222146
Validation Pixel Acc: 0.7515917897571976


Finish epoch 3, time elapsed 28.221797227859497
Train Avg Loss: 2.430846535733768
Validation Loss: 1.7209132313728333
Validation IoU: 0.048834528824204484
Validation Pixel Acc: 0.7424669449252915


Finish epoch 4, time elapsed 28.19951868057251
Train Avg Loss: 2.279164915425437
Validation Loss: 1.5418122070176261
Validation IoU: 0.04855868898169841
Validation Pixel Acc: 0.7448361766581632


Finish epoch

In [27]:
modelTest()

Test Loss at Test: 1.459080423627581
Test IoU at Test: 0.04730005490900689
Test Pixel acc at Test: 0.7262837490604499
